In [ ]:
# default_exp face_class_ver

In [1]:
import os
os.chdir('..')
os.getcwd()

'/home/ubuntu/hw2p2'

In [2]:
#hide
%load_ext autoreload
%autoreload 2

In [3]:
#export
# imports
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision
from PIL import Image
from torch.nn.functional import cosine_similarity, adaptive_avg_pool2d
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import roc_auc_score
import pandas as pd
from functools import partial
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from hyperopt.pyll.base import scope
import json
from datetime import datetime
import os
import time

# from datasets import FaceClassificationDataset, FaceVerificationDataset
# from losses import CenterLoss
# from models.mobilenet import *
# from models.resnet import *
# from hyperoptimization import fit_predict
from hw2p2.datasets import FaceClassificationDataset, FaceVerificationDataset
from hw2p2.losses import CenterLoss
from hw2p2.models.mobilenet import *
from hw2p2.models.resnet import *
from hw2p2.hyperoptimization import fit_predict

In [10]:
#export
def main(model, batch_size, sample_size=None, max_evals=20):

    # Hyperparameters space
    space = {'model': hp.choice(label='model', options=[model]),
             'in_channels': hp.choice(label='in_channels', options=[3]),
            #  'n_classes': hp.choice(label='n_classes', options=[4000]),
             'batch_size': scope.int(hp.choice(label='batch_size', options=[batch_size])),
             'lr': hp.loguniform(label='lr', low=np.log(5e-4), high=np.log(5e-2)),
             'lr_decay': hp.choice(label='lr_decay', options=[0.5]),
             'n_lr_decay_steps': hp.choice(label='n_lr_decay_steps', options=[3]),
             'center_loss': hp.choice(label='center_loss', options=[True]),
             'lr_cl': hp.loguniform(label='lr_cl', low=np.log(5e-4), high=np.log(2e-1)),
             'alpha_cl': hp.choice(label='alpha_cl', options=[0.01,0.1,1]),
             'n_epochs': hp.choice(label='n_epochs', options=[100]),
             'eval_steps': scope.int(hp.choice(label='eval_steps', options=[4])),}

    if sample_size==None:
        space['n_classes'] = hp.choice(label='n_classes', options=[4000])
    else:
        space['n_classes'] = hp.choice(label='n_classes', options=[sample_size])

    # Hyperparameters search
    trials = Trials()
    fmin_objective = partial(fit_predict, verbose=True, trials=trials, sample_size=sample_size)
    fmin(fmin_objective, space=space, 
         algo=tpe.suggest, max_evals=max_evals, trials=trials)

In [11]:
os.getcwd()
# os.chdir('..')

'/home/ubuntu/hw2p2'

In [12]:
#export
if __name__ == "__main__": 
    main(model='resnet18', batch_size=128, sample_size=200, max_evals=20)

                                                      
Current directory: /home/ubuntu/hw2p2

17-03-21_20-16-07                                     
alpha_cl                 0.1                          
batch_size               128
center_loss             True
eval_steps                 4
in_channels                3
lr                  0.047087
lr_cl                    0.1
lr_decay                   1
model               resnet18
n_classes                200
n_epochs                 100
n_lr_decay_steps           1
dtype: object

train_dataset_len: 19037, val_c_dataset_len: 400, val_v_dataset_len: 200
==============================Start Fitting==============================
epoch: 0 train_loss: 49.0981 train_c_acc: 1.18% val_c_loss: 56.6439 val_c_acc: 0.52% val_v_acc: 59.88% 
epoch: 4 train_loss: 17.6656 train_c_acc: 3.43% val_c_loss: 57.2275 val_c_acc: 2.08% val_v_acc: 64.26% 
epoch: 8 train_loss: 9.1662 train_c_acc: 4.65% val_c_loss: 56.414 val_c_acc: 3.12% val_v_acc: 65.59% 
epoch

KeyboardInterrupt: 